# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from datetime import date
%matplotlib inline

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df_path = "output_data/city-df.csv"


# Read the mouse data and the study results
city_data = pd.read_csv(city_df_path)
city_data


,City,Country,Date,Latitude,Longitude,Humidity,Temperature,Temp Max,Cloudiness,Wind Speed
0,qaanaaq,GL,1.624186e+09,77.4840,-69.3632,63.0,36.16,36.16,0.0,5.14
1,pitimbu,BR,1.624186e+09,-7.4706,-34.8086,78.0,72.19,72.19,29.0,7.27
2,cape town,ZA,1.624186e+09,-33.9258,18.4232,25.0,79.88,82.11,0.0,14.97
3,mataura,NZ,1.624186e+09,-46.1927,168.8643,98.0,38.48,38.48,53.0,3.27
4,egvekinot,RU,1.624186e+09,66.3167,-179.1667,53.0,51.44,51.44,21.0,6.69
...,...,...,...,...,...,...,...,...,...,...
525,iquique,US,1.624186e+09,39.4470,-86.1758,93.0,71.24,73.90,55.0,7.20
526,kuryk,MV,1.624186e+09,5.4444,73.3657,73.0,82.83,82.83,51.0,15.99
527,kathmandu,BR,1.624186e+09,-20.3297,-40.2925,83.0,69.75,69.75,75.0,5.75
528,chama,RU,1.624186e+09,55.4888,60.8767,36.0,74.89,74.89,8.0,18.52


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
city_data = city_data

humidity = city_data["Humidity"].astype(float)

In [6]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
city_ideal = city_data.loc[(city_data["Temperature"] < 80) & (city_data["Temperature"] > 70) & (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]
city_ideal

,City,Country,Date,Latitude,Longitude,Humidity,Temperature,Temp Max,Cloudiness,Wind Speed
22,victoria,ZM,1.624186e+09,-16.1167,23.2667,25.0,78.64,78.64,0.0,8.43
377,harrison,KZ,1.624186e+09,43.0369,74.7114,31.0,73.44,73.44,0.0,6.71
456,leninskoye,ZM,1.624186e+09,-13.6202,29.3939,34.0,72.01,72.01,0.0,9.71
501,amahai,MA,1.624186e+09,31.6315,-8.0083,49.0,73.47,73.47,0.0,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = city_ideal.copy()
hotel_df['Hotel Name'] = ""

hotel_df

,City,Country,Date,Latitude,Longitude,Humidity,Temperature,Temp Max,Cloudiness,Wind Speed,Hotel Name
22,victoria,ZM,1.624186e+09,-16.1167,23.2667,25.0,78.64,78.64,0.0,8.43,
377,harrison,KZ,1.624186e+09,43.0369,74.7114,31.0,73.44,73.44,0.0,6.71,
456,leninskoye,ZM,1.624186e+09,-13.6202,29.3939,34.0,72.01,72.01,0.0,9.71,
501,amahai,MA,1.624186e+09,31.6315,-8.0083,49.0,73.47,73.47,0.0,5.75,


In [9]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 22: victoria.
Closest hotel in victoria is Kashuwa Lodge.
--------------------Ending Search
Retrieving Results for Index 377: harrison.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 456: leninskoye.
Closest hotel in leninskoye is HOTEL MARIANA.
--------------------Ending Search
Retrieving Results for Index 501: amahai.
Closest hotel in amahai is Sofitel Marrakech Lounge & Spa.
--------------------Ending Search


In [11]:
hotel_df

,City,Country,Date,Latitude,Longitude,Humidity,Temperature,Temp Max,Cloudiness,Wind Speed,Hotel Name
22,victoria,ZM,1.624186e+09,-16.1167,23.2667,25.0,78.64,78.64,0.0,8.43,Kashuwa Lodge
377,harrison,KZ,1.624186e+09,43.0369,74.7114,31.0,73.44,73.44,0.0,6.71,
456,leninskoye,ZM,1.624186e+09,-13.6202,29.3939,34.0,72.01,72.01,0.0,9.71,HOTEL MARIANA
501,amahai,MA,1.624186e+09,31.6315,-8.0083,49.0,73.47,73.47,0.0,5.75,Sofitel Marrakech Lounge & Spa


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))